In [1]:
import numpy as np
import pandas as pd
import os
import gc
import restructure_data
import feature_engineer
import modeling

In [11]:
import time

In [12]:
time.sleep(5)
print(1)

1


In [2]:
import importlib
# importlib.reload(feature_engineer)
# importlib.reload(modeling)

In [13]:
for i in range(42):
    submit_filepath = './submit/submet_01.csv'
    if os.path.isfile(submit_filepath) == True:
        df_submit = pd.read_csv(submit_filepath)
        print(f'Read submit file. {df_submit.shape}')
    else:
        df_submit = pd.DataFrame(columns=['chid', 'top1', 'top2', 'top3'])
        print('Create submit file.')
    start_index = df_submit.shape[0]
    print(start_index)
    data = restructure_data.DataRestructuring(df_train_path=r'D:\Github\07_E_SUN_2021_Winter_AI\data\tbrain_cc_training_48tags_hash_final.csv',
                                                df_test_path=r'D:\Github\07_E_SUN_2021_Winter_AI\data\需預測的顧客名單及提交檔案範例.csv',
                                                start_index=start_index,
                                                number=10000)
    data.execute()
    time.sleep(5)
    fe = feature_engineer.FeatureEngineering(matrix=data.matrix)
    fe.execute()
    time.sleep(5)
    del data
    gc.collect()
    fit_model = modeling.Model(matrix=fe.matrix)
    fit_model.execute()
    df_submit = pd.concat([df_submit, fit_model.df_result]
                          ).reset_index(drop=True)
    df_submit.to_csv(submit_filepath, index=0)
    del fe, fit_model
    gc.collect()
    time.sleep(10)

Read submit file. (70000, 4)
70000
Iteration is stopped.
Memory usage of dataframe is 46.35 MB
Memory usage after optimization is: 13.18 MB
Decreased by 71.6%
Memory usage of dataframe is 358.58 MB
Memory usage after optimization is: 228.88 MB
Decreased by 36.2%
Adding feature shop_tag_txn_amt_mean_delta_1_lag_1
Adding feature shop_tag_txn_amt_mean_delta_1_lag_12
Adding feature chid_txn_amt_mean_delta_1_lag_1
Adding feature chid_txn_amt_mean_delta_1_lag_12
Adding feature educd_txn_amt_mean_delta_1_lag_1
Adding feature trdtp_txn_amt_mean_delta_1_lag_1
Adding feature gender_code_txn_amt_mean_delta_1_lag_1
Adding feature age_txn_amt_mean_delta_1_lag_1
Creating feature "chid_shop_tag_txn_amt_mean_rolling_mean_win_12"
Creating feature "chid_shop_tag_txn_amt_mean_expanding_mean"
Creating feature "chid_shop_tag_txn_amt_mean_ewm_hl_1"
Creating feature "age_shop_tag_txn_amt_mean_rolling_mean_win_12"
Creating feature "age_shop_tag_txn_amt_mean_expanding_mean"
Creating feature "age_shop_tag_txn_a

In [18]:
submit_filepath = './submit/submet_01.csv'
if os.path.isfile(submit_filepath) == True:
    df_submit = pd.read_csv(submit_filepath)
    print(f'Read submit file. {df_submit.shape}')
else:
    df_submit = pd.DataFrame(columns=['chid', 'top1', 'top2', 'top3'])
    print('Create submit file.')

Read submit file. (490000, 4)


In [19]:
# i = 4
start_index = df_submit.shape[0]
print(start_index)

490000


In [20]:
data = restructure_data.DataRestructuring(df_train_path=r'D:\Github\07_E_SUN_2021_Winter_AI\data\tbrain_cc_training_48tags_hash_final.csv',
                                          df_test_path=r'D:\Github\07_E_SUN_2021_Winter_AI\data\需預測的顧客名單及提交檔案範例.csv',
                    start_index=start_index,
                    number=10000)
data.execute()


Iteration is stopped.
Memory usage of dataframe is 47.09 MB
Memory usage after optimization is: 14.69 MB
Decreased by 68.8%
Memory usage of dataframe is 343.32 MB
Memory usage after optimization is: 228.88 MB
Decreased by 33.3%


In [25]:
data.matrix[data.matrix.poscd.isna()]


,dt,chid,shop_tag,txn_cnt,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,month
1856,1,10305553,2,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
1857,1,10305553,6,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
1858,1,10305553,10,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
1859,1,10305553,12,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
1860,1,10305553,13,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999163,25,10222411,26,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
3999164,25,10222411,36,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
3999165,25,10222411,37,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2
3999166,25,10222411,39,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2


In [35]:
data.matrix[np.isinf(data.matrix.txn_amt)]


,dt,chid,shop_tag,txn_cnt,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,month
1738974,11,10422345,39,144,inf,1.0,4.0,14.0,1.0,1.0,30.0,609516.5625,0.0,4.0,1.0,12


In [26]:
fe = feature_engineer.FeatureEngineering(matrix=data.matrix)


In [27]:
fe._month_feature_engineering()


In [28]:
fe._shop_tag_feature_engineering()


In [29]:
fe._cluster_feature_engineering()


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [30]:
matrix = data.matrix
start_month = 12
end_month = 24
pt = matrix.query(f"dt>{start_month} & dt<={end_month}")
if [] is not None:
    pt = matrix[~matrix['shop_tag'].isin([])]


In [31]:
pt = pt.pivot_table(values='txn_amt', columns='shop_tag',
                    index='dt', fill_value=0, aggfunc='mean')


In [32]:
pt

shop_tag,2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48
dt,,,,,,,,,,,,,,,,
1,1455.309570,446.070679,1348.674561,663.883118,47.485474,1252.133545,373.382050,716.058411,339.757324,212.542725,361.961884,355.676178,859.449219,2416.573975,5.208010e+02,1175.013428
2,1519.572510,467.810242,1296.367065,720.948975,52.325283,1314.516235,368.701477,784.084045,280.587982,224.824966,380.828949,394.395630,1015.034851,2425.310791,1.188281e+24,1150.913452
3,1481.060059,447.849243,1390.273560,764.215210,68.381371,1398.010132,399.934174,834.248718,308.408386,223.444107,387.877502,439.815460,1094.785156,2562.771240,5.466971e+02,1149.571167
4,1579.094116,503.969513,1404.483521,706.283325,61.290604,1468.103638,373.550781,860.574890,338.438599,277.077148,406.665741,383.890411,1067.467529,2767.309814,2.417085e+05,1200.688110
5,1527.686279,537.929504,1492.842896,789.390503,68.412254,1622.498535,433.900330,947.312256,350.700775,256.072754,422.459991,410.891632,1152.323975,2943.778076,7.295134e+02,1267.168091
6,1698.305176,528.302063,1631.951904,799.351624,137.980530,1762.345825,420.074768,927.344360,363.107391,264.671844,410.435547,475.561157,1219.652344,3344.140137,7.020577e+02,1302.425537
7,1722.584839,516.579895,1748.871216,800.048401,179.689011,1693.382812,534.046143,871.491333,386.608643,271.699249,436.569183,394.324036,1274.333496,2936.147705,6.272000e+02,1337.049805
8,2385.602539,552.329773,1823.463867,775.169128,229.792801,1758.304443,508.634186,1038.337891,329.528015,262.348755,439.453125,519.476685,1452.517456,2841.618408,3.211602e+03,1394.540894
9,2145.671631,572.172852,2402.989990,911.112427,251.285324,1838.747192,545.058289,1122.813599,349.746948,277.992371,471.819702,514.900635,1489.267090,2997.119873,1.068822e+18,1396.297729


In [23]:
fe = feature_engineer.FeatureEngineering(matrix=data.matrix)
fe.execute()


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [6]:
del data
gc.collect()
fit_model = modeling.Model(matrix=fe.matrix)
fit_model.execute()

Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 2715.59	training's l2: 7.37443e+06	valid_1's rmse: 2917.95	valid_1's l2: 8.51444e+06
[200]	training's rmse: 2587.66	training's l2: 6.69599e+06	valid_1's rmse: 2833.32	valid_1's l2: 8.02771e+06
[300]	training's rmse: 2547.83	training's l2: 6.49144e+06	valid_1's rmse: 2822.74	valid_1's l2: 7.96788e+06
[400]	training's rmse: 2523.51	training's l2: 6.36811e+06	valid_1's rmse: 2821.64	valid_1's l2: 7.96164e+06
Early stopping, best iteration is:
[394]	training's rmse: 2524.79	training's l2: 6.37459e+06	valid_1's rmse: 2821.46	valid_1's l2: 7.96064e+06


In [7]:
df_submit = pd.concat([df_submit, fit_model.df_result]).reset_index(drop=True)
df_submit.to_csv(submit_filepath, index=0)

In [8]:
del fe, fit_model
gc.collect()

40

In [14]:
df_submit

,chid,top1,top2,top3
0,10000148,37,2,10
1,10000172,10,15,2
2,10000226,37,2,15
3,10000229,15,10,48
4,10000285,2,18,10
...,...,...,...,...
489995,10499816,15,48,37
489996,10499841,48,10,37
489997,10499873,2,36,15
489998,10499878,37,10,2
